# Towers Of Hanoi 

## The model

In [40]:
from towers_game import Tower
from agent import Agent
import torch
import numpy as np
import random

disk_amount = 5

agent = Agent(disk_amount)
tower = Tower(disk_amount)

model_state = torch.load("/Users/shawnwei/Documents/GitHub/Data-Science-EOL/towers_of_hanoi/models/best_agent_5.pth", map_location="cpu")
agent.move_model.load_state_dict(model_state)
agent.move_model.eval()

solved = False
state = tower.get_initial_state()
tower_state = state[0].unsqueeze(0)

move_state_pair = []

while not solved:
    move = agent.pick_move(tower_state)
    move_state_pair.append([tower_state, move])
    state = tower.step(move[0],move[1])
    agent.add_points(state[1])
    tower_state = state[0].unsqueeze(0)
    print(tower.tower)
    if tower.completed():
        agent.way_of_end = 1
        completed = True
        break
    elif agent.is_dead():
        agent.way_of_end = -1
        break

[[5, 4, 3, 2], [], [1]]
[[5, 4, 3], [2], [1]]
[[5, 4, 3], [2, 1], []]
[[5, 4], [2, 1], [3]]
[[5, 4, 1], [2], [3]]
[[5, 4, 1], [], [3, 2]]
[[5, 4], [], [3, 2, 1]]
[[5], [4], [3, 2, 1]]
[[5], [4, 1], [3, 2]]
[[5, 2], [4, 1], [3]]
[[5, 2, 1], [4], [3]]
[[5, 2, 1], [4, 3], []]
[[5, 2], [4, 3], [1]]
[[5], [4, 3, 2], [1]]
[[5], [4, 3, 2, 1], []]
[[], [4, 3, 2, 1], [5]]
[[1], [4, 3, 2], [5]]
[[1], [4, 3], [5, 2]]
[[], [4, 3], [5, 2, 1]]
[[3], [4], [5, 2, 1]]
[[3], [4, 1], [5, 2]]
[[3, 2], [4, 1], [5]]
[[3, 2, 1], [4], [5]]
[[3, 2, 1], [], [5, 4]]
[[3, 2], [], [5, 4, 1]]
[[3], [2], [5, 4, 1]]
[[3], [2, 1], [5, 4]]
[[], [2, 1], [5, 4, 3]]
[[1], [2], [5, 4, 3]]
[[1], [], [5, 4, 3, 2]]
[[], [], [5, 4, 3, 2, 1]]


## This is a custom function used to randomize one specific disk, making its information useless. This is planned to be used to see what the model looks at when trying to determine its next step.

In [54]:
def randomize_disk_segment(state, disk_index):

    # this just incase I change format, also prevent it from changing original
    # needed to reshape this for it to work
    state = np.array(state).copy().reshape(-1) 

    # This find s the locaiton of the disk
    # with three values associated with every disk
    start = disk_index * 3
    end = start + 3
    new_vals = np.random.rand(3)
    state[start:end] = new_vals
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)

    return state

def decode_state(encoded_state, disk_amount):
    encoded = np.array(encoded_state).flatten()
    tower = [[], [], []]
    for disk in range(1, disk_amount + 1):
        start = (disk - 1) * 3
        end = start + 3
        segment = encoded[start:end]
        if np.all(segment == 0):
            continue
        peg = int(np.argmax(segment))
        tower[peg].append(disk)
    for p in range(3):
        tower[p].sort(reverse=True)
    return tower


### Now to demonstrate 

In [59]:
difference = [0 for _ in range(disk_amount)]
important_disk_per_state = []
for pair in range(len(move_state_pair)):
    for disk in range(disk_amount):
        state = randomize_disk_segment(move_state_pair[pair][0], disk)
        move = agent.pick_move(state)
        difference[disk] += move == move_state_pair[pair][1]
        print(disk, decode_state(move_state_pair[pair][0], disk_amount)) if move != move_state_pair[pair][1] and random.random() > .95 else None



2 [[5, 4, 3], [2], [1]]
3 [[5, 4], [], [3, 2, 1]]
4 [[3], [4, 1], [5, 2]]
0 [[3, 2, 1], [4], [5]]
2 [[3, 2], [], [5, 4, 1]]
